In [ ]:
boilerplate = {}
boilerplate_end = {}
codeswitch = {}
codeswitch_end = {}
boilerplate["elin_i_hagen"] = "Elin i hagen by Gustaf Fröding read in Swedish for librivox dot org by Elina Riuttanen"
codeswitch["elin_i_hagen"] = "sv sv sv en sv sv en en en en en en en en fi fi"
boilerplate_end["elin_i_hagen"] = "End of poem. This recording is in the public domain"
boilerplate["en_saga_om_vreden"] = "En saga om Vreden av Fredrika Runeberg uppläst på svenska av Johan Berg. Detta är en LibriVox inspelning. Alla LibriVox inspelningar är allmän egendom. För mer information, eller för att anmäla dig som frivillig, besök librivox punkt org."
boilerplate["efter_torgdagen"] = "Detta är en LibriVox inspelning. Alla LibriVox inspelningar är allmän egendom. För mer information, eller för att anmäla dig som frivillig, besök librivox punkt org. Efter torgdagen av Victoria Benedictsson"


: 

In [1]:
import requests
from mosestokenizer import MosesSentenceSplitter

ModuleNotFoundError: No module named 'requests'

In [11]:
BARON_OLSEN_TITLES = """\
BARON OLSON
TUSEN TJOG ÄGG
ÖMSESIDIGT FÖRTROENDE
»DEN GYLLENE PLOMMONBLOMMAN»
EN RÄTTSFRÅGA
SVÅRSÅLD VARA
ARTIGHET I BARBACKA
JÄRNVÄGSSTREJKEN I FÄLANDA
EN SPRITAFFÄR
MAJUMBA
EN KRIGSLIST
EN ITALIENSK EPISOD
DJURVÄNLIGHET
EN LIVLIG JULAFTON
ETT SJUKDOMSFALL
VÄGEN TILL ENA PIGO
WATERMANS IDEAL
EN HEMSK HISTORIA
MOTORDRIFT
EN BRA MEDICIN
NUTIDA JÄRNVÄGSRESOR
MÖRKSENS GÄRNINGAR
MANNEN SOM VAR EN TIDTABELL
I SNÖSTORMEN
FÖRSTA APRIL
JOURNALISTIK I VILDA VÄSTERN
"""

In [38]:
def runeberg_page(url = "http://runeberg.org/dragharm/elinhage.html", normalise = True, joined = True):
    response = requests.get(url)
    assert response.status_code == 200
    response.encoding = 'UTF-8'
    splitter = "</h1>" if "</h1>" in response.text else "</h3>"
    text = response.text.split(splitter)[1].split("<br clear=all>")[0]
    text = text.replace("<p>", "\n").replace("<br>", "\n")
    if normalise:
        buf = []
        for line in text.split("\n"):
            line = line.strip()
            if line == "":
                continue
            line = line.replace(",", "")
            line = line.replace(" - ", " ")
            if line.startswith("- "):
                line = line[2:]
            if line.endswith(" -"):
                line = line[:-2]
            if line[-1] in ";:.?!":
                line = line[:-1]
            buf.append(line.lower())
    if joined:
        return "\n".join(buf)
    else:
        return buf

def elin_i_hagen(url = "http://runeberg.org/dragharm/elinhage.html", normalise = True):
    return runeberg_page(url, normalise)

def onskenatt(url = "http://runeberg.org/boyemoln/1_16.html", normalise = True):
    return runeberg_page(url, normalise)

def landshovdingen(url = "http://runeberg.org/fstal/2p.html", normalise = True):
    return runeberg_page(url, normalise)

def efter_torgdagen(url = "https://litteraturbanken.se/f%C3%B6rfattare/BenedictssonV/titlar/Ber%C3%A4ttelserOchUtkast/sida/126/etext", normalise = True):
    BASE = url
    return ""

def en_saga_om_vreden(url = "http://web.archive.org/web/20190814032041/http://freetexthost.com:80/bcp31m60i4", normalise = True):
    response = requests.get(url)
    assert response.status_code == 200
    text = response.text.split('<div id="contentsinner">')[1].split('<a href=')[0]
    if normalise:
        text = text.replace("&quot;", "").replace("<br/>", "")
        buf = []
        for line in text.split("\n"):
            if line.strip() == "":
                continue
            #line = line.strip().replace(",", "")
            buf.append(line)
        if buf[0].startswith("En saga om Vreden,"):
            buf = buf[1:]
        with MosesSentenceSplitter('sv') as splitsents:
            sents = splitsents(buf)
        newbuf = []
        for sent in sents:
            sent = sent.replace(",", "")
            sent = sent.replace(" - ", " ")
            if sent.endswith(" -"):
                sent = sent[:-2]
            if sent[-1] in ";:.?!":
                sent = sent[:-1]
            newbuf.append(sent)
        text = "\n".join(newbuf).lower()
    else:
        text = text.replace("&quot;", "\"").replace("<br/>", "\n")
    return text

def baron_olson(url = "https://www.gutenberg.org/cache/epub/15719/pg15719.txt", normalise = True):
    def _skip(text):
        text = text.strip()
        if text == "":
            return True
        if text == "*       *       *       *       *":
            return True
        return False
    stories = {}
    response = requests.get(url)
    assert response.status_code == 200
    GB_START = "*** START OF THIS PROJECT GUTENBERG EBOOK BARON OLSON OCH ANDRA HISTORIER ***"
    GB_END = "End of Project Gutenberg's Baron Olson och andra historier, by Sigge Strömberg"
    text = response.text.split(GB_START)[1].split(GB_END)[0]
    chapter_titles = [t for t in BARON_OLSEN_TITLES.split("\n") if t != ""]
    prev_title = ""
    cur_title = chapter_titles.pop(0)
    text_buf = []
    for line in text.replace("\r", "").split("\n"):
        if line.strip() == cur_title:
            if prev_title != "":
                stories[prev_title] = text_buf.copy()
            text_buf.clear()
            prev_title = cur_title
            cur_title = chapter_titles.pop(0)
        elif not _skip(line):
            text_buf.append(line)
    return stories

def tomten(url = "https://www.gutenberg.org/cache/epub/15753/pg15753.txt", normalise = True):
    import re
    response = requests.get(url)
    assert response.status_code == 200
    text = response.text.split("TOMTEN.")[1].split("I NATTEN.")[0]
    text = text.replace("\r", "")
    buf = []
    for line in text.split("\n"):
        line = line.strip()
        if line == "":
            continue
        line = re.sub(" +[1-8]?[05]$", "", line)
        if normalise:
            line = line.replace(",", "")
            line = line.replace("--", " ")
            line = line.replace('"', "")
            if line[-1] in ";:.?!":
                line = line[:-1]
            line = line.lower()
        buf.append(line)
    text = "\n".join(buf)
    print(text)